In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,

In [2]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'cry no_cry'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'b_cry/{g}'):
        songname = f'b_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=7)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 576x576 with 0 Axes>

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 16):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [4]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'cry no_cry'.split()
for g in genres:
    for filename in os.listdir(f'b_cry/{g}'):
        songname = f'b_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [5]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [7]:
classifier = model.fit(X_train,y_train,shuffle=True,epochs=50,batch_size=128)

Epoch 1/50
2/2 [==============================] - ETA: 0s - loss: 2.5732 - accuracy: 0.0000e+ - 0s 5ms/step - loss: 2.5591 - accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 2.1181 - accuracy: 0.29 - 0s 4ms/step - loss: 2.1123 - accuracy: 0.3111
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.8005 - accuracy: 0.82 - 0s 6ms/step - loss: 1.7920 - accuracy: 0.8148
Epoch 4/50
2/2 [==============================] - ETA: 0s - loss: 1.5236 - accuracy: 0.82 - 0s 5ms/step - loss: 1.5136 - accuracy: 0.8222
Epoch 5/50
2/2 [==============================] - ETA: 0s - loss: 1.2534 - accuracy: 0.82 - 0s 6ms/step - loss: 1.2501 - accuracy: 0.8222
Epoch 6/50
2/2 [==============================] - ETA: 0s - loss: 1.0174 - accuracy: 0.83 - 0s 5ms/step - loss: 1.0094 - accuracy: 0.8444
Epoch 7/50
2/2 [==============================] - ETA: 0s - loss: 0.8080 - accuracy: 0.85 - 0s 5ms/step - loss: 0.8065 - accuracy: 0.8519
Epoch 8/50
2/2 [==========

In [8]:
model.evaluate_generator(generator=X_test, steps=50)#OUTPUT

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.0, 0.0]

In [70]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [75]:
# cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
pathlib.Path(f'live_data/').mkdir(parents=True, exist_ok=True)
# for filename in os.listdir(f'b_cry/{g}'):
songname = f'output.wav'
y, sr = librosa.load(songname, mono=True, duration=7)
plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
plt.axis('off');
plt.savefig(f'live_data/output.png')
plt.clf()

<Figure size 576x576 with 0 Axes>

In [76]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 16):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [77]:
file = open('predict.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
# genres = 'discomfort belly_pain burping hungry'.split()
# for g in genres:
#     for filename in os.listdir(f'b_cry/{g}'):
# songname = f'b_cry/{g}/{filename}'
songname = f'output.wav'
y, sr = librosa.load(songname, mono=True, duration=5)
rmse = librosa.feature.rms(y=y)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
to_append = f'output.wav {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'   
for e in mfcc:
    to_append += f' {np.mean(e)}'
# to_append += f' {model.predict(live_data.iloc[0])}'
#to_append += f' live_data_genre'
file = open('predict.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())
live_data = pd.read_csv('predict.csv')
live_data = live_data.drop(['filename'],axis=1)


In [78]:
#print(live_data.iloc[0][0:26])
Xnew = np.array([live_data.iloc[0][0:21]])
print(Xnew)
ynew = model.predict_classes(Xnew)
print("Predicted=%s" % (ynew))
#live_data = data.drop(['filename'],axis=1)
#print(model.predict(live_data.iloc[:, -1]))
# print(type(live_data.iloc[0]))
#print(model.predict_classes(live_data.iloc[0]))
# print(live_data.iloc[0])

[[ 2.49939677e+03  1.24744557e-01 -2.78122620e+02  1.31338074e+02
  -8.87330933e+01 -4.91475630e+00  8.92387199e+00 -5.33820877e+01
  -6.17922878e+00  8.00373936e+00 -1.41683397e+01  1.10049124e+01
  -1.08737662e-01 -1.57673702e+01  7.95671320e+00 -2.77607751e+00
  -1.21360607e+01  8.09323883e+00  8.44810069e-01 -3.53708744e+00
   7.11395216e+00]]
Predicted=[1]


In [45]:
#p_pred=model.predict_proba(X_test)

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1], dtype=int64)

In [40]:
y_pred=model.predict_classes(X_test)
y_pred

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [27]:
y_test=y_test.astype('int64')
y_test

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [28]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.9411764705882353

In [29]:
asd=model.predict_classes(np.array([live_data.iloc[0][0:21]]))
asd

array([1], dtype=int64)